In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
training_data = datasets.FashionMNIST(root='data', download=True, train=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='data', download=True, train=False, transform=transforms.ToTensor())

In [5]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
len(train_dataloader), len(test_dataloader), 938*64, 157*64

(938, 157, 60032, 10048)

In [7]:
len(train_dataloader.dataset), len(test_dataloader.dataset), len(test_dataloader.dataset) // 64, len(test_dataloader)

(60000, 10000, 156, 157)

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [9]:
learning_rate, epochs, batch_size = 1e-3, 5, 64

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss:>7f}  [{current:>5d} / {size:>5d}]")

In [15]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error :\nAccuracy:{(100 * correct):>0.1f}%, Avg loss : {test_loss:>8f}\n")

In [16]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------------------------------
loss : 2.494542  [    0 / 60000]
loss : 0.569003  [ 6400 / 60000]
loss : 0.414040  [12800 / 60000]
loss : 0.521176  [19200 / 60000]
loss : 0.449371  [25600 / 60000]
loss : 0.411929  [32000 / 60000]
loss : 0.313210  [38400 / 60000]
loss : 0.473859  [44800 / 60000]
loss : 0.404665  [51200 / 60000]
loss : 0.429685  [57600 / 60000]
Test Error :
Accuracy:85.0%, Avg loss : 0.409072

Epoch 2
-----------------------------------------------------------
loss : 0.379758  [    0 / 60000]
loss : 0.333252  [ 6400 / 60000]
loss : 0.249364  [12800 / 60000]
loss : 0.318370  [19200 / 60000]
loss : 0.342015  [25600 / 60000]
loss : 0.338753  [32000 / 60000]
loss : 0.265624  [38400 / 60000]
loss : 0.391231  [44800 / 60000]
loss : 0.319767  [51200 / 60000]
loss : 0.342396  [57600 / 60000]
Test Error :
Accuracy:85.9%, Avg loss : 0.380218

Epoch 3
-----------------------------------------------------------
loss : 0.274174  [    0 / 60000]
los